In [3]:
import os
# ptb 모듈 경로 추가
import sys
sys.path.append('C:/Users/KimDongyoung/Desktop/Github/my_git/mygit/DEEPLEARNING/밑바닥부터시작하는딥러닝2')

In [4]:
from sequence import load_data, get_vocab
from common.optimizer import Adam
from common.trainer import Trainer
from common.util import eval_seq2seq
from ch07.seq2seq import Seq2seq
import numpy as np
from common.layers import *

In [5]:
# 데이터셋 읽기
(x_train, t_train), (x_test, t_test) = load_data('addition.txt')
char_to_id, id_to_char = get_vocab()

# 하이퍼파라미터 설정
vocab_size = len(char_to_id)
wordvec_size = 16 # 단어 임베딩 차원수
hidden_size = 128
batch_size = 128
max_epoch = 25
max_grad = 5.0

# 모델, 옵티마이저, 트레이너 생성
model = Seq2seq(vocab_size, wordvec_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

In [6]:
x_train.shape # 45000개의 데이터가 있고, 각 데이터는 7개의 문자로 이루어져 있다. 문자 = 단어 ID

(45000, 7)

In [7]:
x_train[0]

array([ 3,  0,  2,  0,  0, 11,  5])

In [8]:
acc_list = [] # 정확도를 저장할 리스트

for epoch in range(max_epoch):
  trainer.fit(x_train, t_train, max_epoch=1, batch_size=batch_size, max_grad=max_grad)
  correct_num = 0
  
  for i in range(len(x_test)): # 테스트 데이터를 이용해 평가, len(x_test)는 테스트 데이터의 개수 = 5000
    question, correct = x_test[[i]], t_test[[i]]
    verbose = i < 10 # 최초 10개만 출력
    correct_num += eval_seq2seq(model, question, correct, id_to_char, verbose)
  
  acc = float(correct_num) / len(x_test)
  acc_list.append(acc)
  print('정확도 %.3f%%' % (acc * 100))

| 에폭 1 |  반복 1 / 351 | 시간 0[s] | 손실 2.56
| 에폭 1 |  반복 21 / 351 | 시간 0[s] | 손실 2.53
| 에폭 1 |  반복 41 / 351 | 시간 0[s] | 손실 2.17
| 에폭 1 |  반복 61 / 351 | 시간 1[s] | 손실 1.96
| 에폭 1 |  반복 81 / 351 | 시간 1[s] | 손실 1.92
| 에폭 1 |  반복 101 / 351 | 시간 2[s] | 손실 1.87
| 에폭 1 |  반복 121 / 351 | 시간 2[s] | 손실 1.85
| 에폭 1 |  반복 141 / 351 | 시간 3[s] | 손실 1.83
| 에폭 1 |  반복 161 / 351 | 시간 3[s] | 손실 1.79
| 에폭 1 |  반복 181 / 351 | 시간 3[s] | 손실 1.77
| 에폭 1 |  반복 201 / 351 | 시간 4[s] | 손실 1.77
| 에폭 1 |  반복 221 / 351 | 시간 4[s] | 손실 1.76
| 에폭 1 |  반복 241 / 351 | 시간 5[s] | 손실 1.76
| 에폭 1 |  반복 261 / 351 | 시간 5[s] | 손실 1.76
| 에폭 1 |  반복 281 / 351 | 시간 6[s] | 손실 1.75
| 에폭 1 |  반복 301 / 351 | 시간 6[s] | 손실 1.74
| 에폭 1 |  반복 321 / 351 | 시간 6[s] | 손실 1.75
| 에폭 1 |  반복 341 / 351 | 시간 7[s] | 손실 1.74
Q 77+85  
T 162 
X 100 
---
Q 975+164
T 1139
X 1000
---
Q 582+84 
T 666 
X 1000
---
Q 8+155  
T 163 
X 100 
---
Q 367+55 
T 422 
X 1000
---
Q 600+257
T 857 
X 1000
---
Q 761+292
T 1053
X 1000
---
Q 830+597
T 1427
X 1000
---
Q 26+838

In [9]:
len(x_test)

5000

In [10]:
class TimeEmbedding:
    def __init__(self, W):
        self.params = [W] # 단어 ID에 해당하는 단어 벡터를 한꺼번에 저장
        self.grads = [np.zeros_like(W)]
        self.layers = None
        self.W = W

    def forward(self, xs):
        N, T = xs.shape
        V, D = self.W.shape

        out = np.empty((N, T, D), dtype='f')
        self.layers = []

        for t in range(T):
            layer = Embedding(self.W)
            out[:, t, :] = layer.forward(xs[:, t])
            self.layers.append(layer)

        return out

    def backward(self, dout):
        N, T, D = dout.shape

        grad = 0
        for t in range(T):
            layer = self.layers[t]
            layer.backward(dout[:, t, :])
            grad += layer.grads[0]

        self.grads[0][...] = grad
        return None

In [11]:
# 예시 데이터
vocab_size = 6  # 어휘 크기
embed_size = 10    # 임베딩 벡터의 차원
W = np.random.randn(vocab_size, embed_size)  # 임베딩 행렬

# TimeEmbedding 계층 생성
time_embedding = TimeEmbedding(W)

# 입력 시퀀스 (배치 크기, 시간 단계 수)
xs = np.array([[1, 2, 3], [4, 5, 2]]) # 숫자는 단어 ID라고 가정

# 순전파
out = time_embedding.forward(xs)
print("입력 시퀀스의 형태:", xs.shape)
print("임베딩 벡터의 형태:", out.shape)

입력 시퀀스의 형태: (2, 3)
임베딩 벡터의 형태: (2, 3, 10)


In [12]:
xs.shape

(2, 3)

In [13]:
time_embedding.params

[array([[-0.63849482, -1.32506091,  0.5075684 , -0.11935398, -0.51599534,
          0.08598084,  1.64906016,  0.8787836 , -0.3368146 , -0.72172928],
        [-1.40644744,  1.24293544,  1.02691155, -0.39824612,  1.489959  ,
          0.19939475, -0.97076312, -0.55643303,  0.34388132, -1.08087195],
        [-0.90257912,  0.42967783,  0.71144932, -0.3335634 ,  1.0436824 ,
          1.42282492, -1.14696863,  0.8775764 , -1.29573916, -0.94019079],
        [-0.55417796,  0.82836573,  0.54420414,  0.64401917, -0.74964611,
         -0.66890052, -0.12513328,  0.01442898, -0.22536117,  1.39998723],
        [-1.25966619, -0.43431685, -0.83531086,  0.67833703, -0.21239583,
          1.16722147, -0.2925453 , -1.26268603,  0.37736995, -0.61400201],
        [-0.57275916,  1.7594075 , -1.31400414, -0.17305501,  0.25553431,
          0.44794741,  0.03219756, -0.4222679 ,  0.28547711,  0.64798971]])]

In [14]:
out

array([[[-1.4064474 ,  1.2429354 ,  1.0269115 , -0.3982461 ,
          1.489959  ,  0.19939475, -0.97076315, -0.556433  ,
          0.3438813 , -1.0808719 ],
        [-0.9025791 ,  0.42967784,  0.7114493 , -0.3335634 ,
          1.0436825 ,  1.4228249 , -1.1469686 ,  0.8775764 ,
         -1.2957392 , -0.9401908 ],
        [-0.55417794,  0.82836574,  0.5442041 ,  0.6440192 ,
         -0.7496461 , -0.6689005 , -0.12513328,  0.01442898,
         -0.22536117,  1.3999872 ]],

       [[-1.2596662 , -0.43431684, -0.8353109 ,  0.67833704,
         -0.21239583,  1.1672214 , -0.2925453 , -1.262686  ,
          0.37736994, -0.614002  ],
        [-0.57275915,  1.7594075 , -1.3140042 , -0.17305501,
          0.2555343 ,  0.4479474 ,  0.03219756, -0.4222679 ,
          0.2854771 ,  0.6479897 ],
        [-0.9025791 ,  0.42967784,  0.7114493 , -0.3335634 ,
          1.0436825 ,  1.4228249 , -1.1469686 ,  0.8775764 ,
         -1.2957392 , -0.9401908 ]]], dtype=float32)